In [ ]:
##Data Preparation
### Skin_disease Dataset (custom data) 다운로드
##### Public Dataset : https://universe.roboflow.com/skin-disease-detetcion/skin-disease-prediction

# !wget -O Skin_disease.zip https://universe.roboflow.com/ds/hRg5MsBht5?key=CsLBs48ZJk

import zipfile

with zipfile.ZipFile('/content/Skin_disease.zip') as target_file:

    target_file.extractall('/content/Skin_disease/')

### yaml 파일 설정 (데이터셋 위치 알려주는 config file)
##### roboflow 에서 제공되는 data.yaml 파일 확인

# !cat /content/Skin_disease/data.yaml

### custom data에 대한 yaml 파일 만들기

# !pip install PyYAML

# yaml 파일을 학습이 가능하도록 경로 설정.
# key-value 데이터인 dict 데이터타입으로 data['train'], data['val'], data['nc'], data['names'] 에 넣어주는데,
# 가장 중요한 부분은 데이터 경로 설정임.

import yaml

data = { 'train' : '/content/Skin_disease/train/images/',
         'val' : '/content/Skin_disease/valid/images/',
         'test' : '/content/Skin_disease/test/images',
         'names' : ['0', '1', '2', '3', '4', '5', '6', '7'],
         'nc' : 8 }

with open('/content/Skin_disease/Skin_disease.yaml', 'w') as f:
  yaml.dump(data, f)


with open('/content/Skin_disease/Skin_disease.yaml', 'r') as f:
  skin_yaml = yaml.safe_load(f)
  display(skin_yaml)

# !cat /content/Skin_disease/Skin_disease.yaml
# !pip install ultralytics

import ultralytics

ultralytics.checks()

# Train
# * 모델 종류는 https://github.com/ultralytics/ultralytics 여기서 확인
#   * yolov8n, yolov8s, yolov8m, yolov8l, yolov8x 등이 있는데, 보통 n,s 정도만 사용하면 충분

# Load YOLOv8n

from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # load a pretrained YOLOv8n detection model

print(type(model.names), len(model.names))

print(model.names)

model.train(data='/content/Skin_disease/Skin_disease.yaml', epochs=150, patience=20, batch=32, imgsz=640)

print(type(model.names), len(model.names))

print(model.names)


### train 과정중에 loss, accuracy 등의 graph 데이터는 runs/detect/train/ 에 있는 results.csv 와 results.png 통해서 확인가능하다
### 테스트 이미지 데이터 생성 및 확인

# 테스트 이미지

from glob import glob

test_image_list = glob('/content/Skin_disease/test/images/*')

print(len(test_image_list))

test_image_list.sort()

for i in range(len(test_image_list)):

    print('i = ',i, test_image_list[i])

## 이미지내의 객체 검출 (Inference)

results = model.predict(source='/content/Skin_disease/test/images/', save=True)
print(type(results), len(results))

import numpy as np

for result in results:

    uniq, cnt = np.unique(result.boxes.cls.cpu().numpy(), return_counts=True)  # Torch.Tensor -> numpy
    uniq_cnt_dict = dict(zip(uniq, cnt))

    print('\n{class num:counts} =', uniq_cnt_dict,'\n')

    for c in result.boxes.cls:
        print('class num =', int(c), ', class_name =', model.names[int(c)])

## 결과 확인 및 다운로드

import glob

detetced_image_list = glob.glob(('/content/runs/detect/predict/*'))

detected_image_nums = len(detetced_image_list)

print(detected_image_nums)

print(detetced_image_list)

# 다운로드를 위한 inference image 압축

import zipfile
import os

if not os.path.exists('/content/detected_result/'):
    os.mkdir('/content/detected_result/')
    print('detected_result dir is created !!!')


with zipfile.ZipFile('/content/detected_result/detected_images.zip', 'w') as detected_images:

    for idx in range(detected_image_nums):
        detected_images.write(detetced_image_list[idx])

from ultralytics import YOLO
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from PIL import Image
import requests
from io import BytesIO

# 이미지 URL
url = "https://images.squarespace-cdn.com/content/v1/5dec67bb31a1cc1ad78a7326/3dd819b7-9bbe-4ec3-9e30-15dd6875c2a0/acne2.png?format=2500w"

# 이미지 다운로드
response = requests.get(url)
img = Image.open(BytesIO(response.content))

def run_img_custom():
  model = YOLO('/content/runs/detect/train/weights/best.pt')
  results = model(img)
  print(results)

  bndboxs = results[0].boxes.data
  class_id = results[0].boxes.cls
  conf = results[0].boxes.conf

  img_array = results[0].orig_img

  names = results[0].names
  print(names)

  for i , bndbox in enumerate(bndboxs):

    xmin = int(bndbox[0])
    ymin = int(bndbox[1])
    xmax = int(bndbox[2])
    ymax = int(bndbox[3])
    conf = float(bndbox[4])
    class_id = int(bndbox[5])
    class_name = names[class_id]

    print(xmin, ymin, xmax, ymax)

    text = f"{class_name}-{round(conf,2)}"

    cv2.rectangle(img_array, (xmin, ymin), (xmax,ymax), (0,255,0),2)
    cv2.putText(img_array, text, (xmin,ymin-5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

  print(text)
  cv2_imshow(img_array)
  cv2.waitKey(0)
  cv2.imwrite("test.png", img_array)

if __name__ == '__main__':
  run_img_custom()

from ultralytics import YOLO
import matplotlib.pyplot as plt

# 모델 초기화
model = YOLO('yolov8n.pt')

# 학습 데이터 설정
data = '/content/Skin_disease/Skin_disease.yaml'

# 에포크, 배치 사이즈, 이미지 크기 설정
epochs = 100
batch_size = 32
image_size = 640

# 손실 기록을 위한 리스트 생성
train_losses = []
val_losses = []

# 조기 종료를 위한 변수 초기화
best_val_loss = float('inf')
patience = 20
early_stop_count = 0

for epoch in range(epochs):
    # 모델 훈련
    model.train(data=data, epochs=1, batch=batch_size, imgsz=image_size)

    # 훈련 데이터의 손실값 기록
    train_loss = model.loss
    train_losses.append(train_loss)

    # 검증 데이터로 손실값 평가
    val_loss = model.evaluate(data=data)['val']['metrics']['loss']
    val_losses.append(val_loss)

    # 학습 과정 중간에 손실값 출력
    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss}, Validation Loss: {val_loss}")

    # 조기 종료 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1
        if early_stop_count >= patience:
            print(f"Early stopping! No improvement for {patience} epochs.")
            break

# 손실값 그래프 시각화
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss with Early Stopping')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()